In [41]:
import pandas as pd

dataset = pd.read_csv("car data.csv")
print(dataset.shape)
dataset.head()

(301, 9)


,Car_Name,Year,Selling_Price,Present_Price,Driven_kms,Fuel_Type,Selling_type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


In [42]:
dataset.drop_duplicates(inplace=True)
y = dataset['Selling_Price']
X = dataset.drop('Selling_Price', axis=1)
print(X.shape, y.shape)

(299, 8) (299,)


In [43]:
cat_col_names = []
num_col_names = []

for col in X.columns:
    if X[col].dtype == 'object':
        cat_col_names.append(col)
    else:
        num_col_names.append(col)

print(cat_col_names)
print(num_col_names)

['Car_Name', 'Fuel_Type', 'Selling_type', 'Transmission']
['Year', 'Present_Price', 'Driven_kms', 'Owner']


In [44]:
for col in cat_col_names:
    print(len(X[col].unique()))

98
3
2
2


In [45]:
cat_col_names = cat_col_names[1:]
print(cat_col_names)

['Fuel_Type', 'Selling_type', 'Transmission']


In [46]:
cat_cols = X[cat_col_names]
num_cols = X[num_col_names]

In [65]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse = False)
cat_cols_OH = pd.DataFrame(encoder.fit_transform(cat_cols))
cat_cols_OH.head()


,0,1,2,3,4,5,6
0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
1,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,1.0,1.0,0.0,0.0,1.0
3,0.0,0.0,1.0,1.0,0.0,0.0,1.0
4,0.0,1.0,0.0,1.0,0.0,0.0,1.0


In [67]:
cat_cols_OH.index = cat_cols.index
OH_col_names = encoder.get_feature_names_out(cat_col_names)
cat_cols_OH.columns = OH_col_names
cat_cols_OH.head()

,Fuel_Type_CNG,Fuel_Type_Diesel,Fuel_Type_Petrol,Selling_type_Dealer,Selling_type_Individual,Transmission_Automatic,Transmission_Manual
0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
1,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,1.0,1.0,0.0,0.0,1.0
3,0.0,0.0,1.0,1.0,0.0,0.0,1.0
4,0.0,1.0,0.0,1.0,0.0,0.0,1.0


In [69]:
X = pd.concat([num_cols, cat_cols_OH], axis=1)
X.head()

,Year,Present_Price,Driven_kms,Owner,Fuel_Type_CNG,Fuel_Type_Diesel,Fuel_Type_Petrol,Selling_type_Dealer,Selling_type_Individual,Transmission_Automatic,Transmission_Manual
0,2014,5.59,27000,0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
1,2013,9.54,43000,0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,2017,9.85,6900,0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
3,2011,4.15,5200,0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
4,2014,6.87,42450,0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


In [73]:
from xgboost import XGBRegressor

model = XGBRegressor(random_state=7)
model.fit(X,y)
predictions = model.predict(X)


from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(predictions, y)
print(mae)

0.013047774993256989
